<a href="https://colab.research.google.com/github/romulomatos/bimaster/blob/master/Numeros_aleatorios_21x25k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import pandas as pd

def gerar_planilha_aleatoria(arquivo_saida="matriz_aleatoria.xlsx",
                             n_linhas=25000,
                             n_colunas=21):
    # Gera matriz de números aleatórios entre 0 e 1
    matriz = np.random.random(size=(n_linhas, n_colunas))

    # Cria nomes de colunas (Col_1, Col_2, ..., Col_21)
    colunas = [f"Col_{i+1}" for i in range(n_colunas)]

    # Converte para DataFrame do pandas
    df = pd.DataFrame(matriz, columns=colunas)

    # Salva em Excel
    df.to_excel(arquivo_saida, index=False)
    print(f"Arquivo '{arquivo_saida}' criado com sucesso!")

    return df


def avaliar_aleatoriedade(df, n_bins=10):
    """
    Faz alguns checks simples de aleatoriedade:
    - Estatísticas descritivas globais
    - Histograma + estatística qui-quadrado vs. uniforme(0,1)
    - Autocorrelação lag-1 média entre linhas de cada coluna
    - Teste de runs (acima/abaixo de 0,5) na sequência linearizada
    """
    valores = df.to_numpy().ravel()
    n = valores.size

    print("\n=== Estatísticas básicas (globais) ===")
    print(f"N total de valores: {n}")
    print(f"Média esperada (teórica): 0.5")
    print(f"Média observada: {valores.mean():.5f}")
    print(f"Desvio-padrão esperado (teórico): ~0.288675")
    print(f"Desvio-padrão observado: {valores.std():.5f}")
    print(f"Valor mínimo observado: {valores.min():.5f}")
    print(f"Valor máximo observado: {valores.max():.5f}")

    # Histograma + qui-quadrado simples
    print("\n=== Histograma e estatística qui-quadrado (aprox.) ===")
    counts, edges = np.histogram(valores, bins=n_bins, range=(0.0, 1.0))
    esperado = n / n_bins
    chi2 = ((counts - esperado) ** 2 / esperado).sum()

    print(f"Número de bins: {n_bins}")
    print("Contagens por bin:", counts)
    print(f"Frequência esperada por bin (uniforme): {esperado:.2f}")
    print(f"Estatística qui-quadrado (sem p-valor): {chi2:.2f}")
    print("Quanto mais próxima a contagem de cada bin estiver do esperado, melhor a aderência à uniforme(0,1).")

    # Autocorrelação lag-1 por coluna
    print("\n=== Autocorrelação lag-1 por coluna ===")
    autocorrs = []
    for col in df.columns:
        v = df[col].values
        v0 = v[:-1] - v[:-1].mean()
        v1 = v[1:] - v[1:].mean()
        denom = np.sqrt((v0 ** 2).sum() * (v1 ** 2).sum())
        if denom == 0:
            ac = 0.0
        else:
            ac = (v0 * v1).sum() / denom
        autocorrs.append(ac)

    autocorrs = np.array(autocorrs)
    for i, ac in enumerate(autocorrs, start=1):
        print(f"Col_{i:02d}: autocorrelação lag-1 = {ac:.5f}")

    print(f"Autocorrelação lag-1 média entre colunas: {autocorrs.mean():.5f}")
    print("Idealmente, valores próximos de 0 indicam pouca dependência linear entre números consecutivos.")

    # Teste de runs (acima/abaixo de 0,5) na sequência completa
    print("\n=== Teste de runs (acima/abaixo de 0,5) ===")
    serie = (valores > 0.5).astype(int)   # 1 se > 0.5, 0 caso contrário
    if len(serie) < 2:
        print("Seqüência muito curta para teste de runs.")
        return

    # Contagem de runs
    runs = 1 + np.sum(serie[1:] != serie[:-1])
    n1 = serie.sum()          # número de 1s (acima de 0,5)
    n2 = len(serie) - n1      # número de 0s (abaixo ou igual a 0,5)

    print(f"Total de runs observados: {runs}")
    print(f"Qtd de valores > 0.5 (n1): {n1}")
    print(f"Qtd de valores <= 0.5 (n2): {n2}")

    if n1 > 0 and n2 > 0:
        # Fórmulas clássicas do teste de runs
        mu_runs = 2 * n1 * n2 / (n1 + n2) + 1
        var_runs = (2 * n1 * n2 * (2 * n1 * n2 - n1 - n2)) / (
            (n1 + n2) ** 2 * (n1 + n2 - 1)
        )
        z_runs = (runs - mu_runs) / np.sqrt(var_runs)

        print(f"Runs esperados (E[R]): {mu_runs:.2f}")
        print(f"Variância de R (Var[R]): {var_runs:.2f}")
        print(f"Estatística Z aproximada: {z_runs:.3f}")
        print("Se |Z| for muito grande (por ex. > 2), pode indicar desvios da aleatoriedade em termos de alternância.")
    else:
        print("Não é possível aplicar o teste de runs (todos valores de um lado de 0,5).")


if __name__ == "__main__":
    # Gera a planilha e obtém o DataFrame
    df = gerar_planilha_aleatoria(
        arquivo_saida="matriz_aleatoria.xlsx",
        n_linhas=25000,
        n_colunas=21
    )

    # Avalia a aleatoriedade dos números gerados
    avaliar_aleatoriedade(df)


Arquivo 'matriz_aleatoria.xlsx' criado com sucesso!

=== Estatísticas básicas (globais) ===
N total de valores: 525000
Média esperada (teórica): 0.5
Média observada: 0.49973
Desvio-padrão esperado (teórico): ~0.288675
Desvio-padrão observado: 0.28867
Valor mínimo observado: 0.00000
Valor máximo observado: 1.00000

=== Histograma e estatística qui-quadrado (aprox.) ===
Número de bins: 10
Contagens por bin: [52522 52611 52941 52055 52236 52769 52489 52557 52443 52377]
Frequência esperada por bin (uniforme): 52500.00
Estatística qui-quadrado (sem p-valor): 10.84
Quanto mais próxima a contagem de cada bin estiver do esperado, melhor a aderência à uniforme(0,1).

=== Autocorrelação lag-1 por coluna ===
Col_01: autocorrelação lag-1 = -0.00323
Col_02: autocorrelação lag-1 = -0.01106
Col_03: autocorrelação lag-1 = 0.00506
Col_04: autocorrelação lag-1 = -0.00700
Col_05: autocorrelação lag-1 = -0.00723
Col_06: autocorrelação lag-1 = -0.01416
Col_07: autocorrelação lag-1 = -0.00538
Col_08: autoco

/tmp/ipython-input-490/1610874692.py:95: RuntimeWarning: overflow encountered in scalar multiply
  var_runs = (2 * n1 * n2 * (2 * n1 * n2 - n1 - n2)) / (
/tmp/ipython-input-490/1610874692.py:98: RuntimeWarning: invalid value encountered in sqrt
  z_runs = (runs - mu_runs) / np.sqrt(var_runs)
